# Attributter i Python

**Attributter** er et begrep om klasser. Attributter er et samlebegrep for

- **Metoder:** funksjoner som er en del av en klasse
- **Datafelt:** variabler som er en del av en klasse

**Identifiser** metoder, funksjoner, klasser, datafelt og instanser i koden under:

In [ ]:
class Line:
    def __init__(self, a, b=0):
        self.a = a
        self.b = b

    def __call__(self, x):
        return self.a*x + self.b

    def root(self):
        return -self.b/self.a


def line(x, a, b=0):
    return a*x + b

c = 2 
x0 = 10
y = Line(c)
print(f"{y(x0) = } & {line(x0, c) = }.")

## *Private* attributter 


Både metoder og datafelt kan være *private*.

- Private datafelt: `self._variable`
- Private metoder: `_method_name(self, args)`

I motsetning til andre programmeringsspråk har ikke Python *ekte* private variabler. Private datafelt og metoder *kan* brukes utenfor klassen, men da er det brukerens ansvar om noe går galt.

### *Setters* og *getters* for private datafelt

- `@property` - Metode som *oppfører seg* som datafelt
- `@field.setter` - Metode som definerer følger av å oppdatere et datafelt

**Kan du komme på noen grunner for å bruke private variabler?**

In [ ]:
class SomeClass:
    def __init__(self, field, private_field):
        self.field = field
        self._private_field = private_field
        
    @property
    def private_field(self):
        return self._private_field
    
    @private_field.setter
    def private_field(self, new_private_field):
        self._private_field = new_private_field
        ...
        
    def method(self, args):
        ...
        
    def _private_method(self, args):
        ...

## Datafelt

Deles inn i to kategorier:
- **Klassevariabler:** Felles for alle instanser av samme klasse og for klassen selv
- **Instansvariabler:** Spesifikk for instans

**Eksempel på klassevariabel og instansvariabel:**

In [ ]:
class Counter:
    count = 0 
    
    def __init__(self):
        Counter.count += 1 
        self.number = self.count       

- `count` er en **klassevariabel**, og kan hentes direkte fra *klassen* `Counter`:

In [ ]:
print(f"count is a class variable: {Counter.count}.")  

- `number` er en **instansvariabel** og er kun definert for instanser av klassen: 

In [ ]:
print(f"number is not a class variable: {Counter.number}.")  

### Overskriving og endringer av klassevariabler

`__init__` endrer `count` ved `Counter.count += 1`.

- Når en **klassevariabel endres, endres den for alle**

In [ ]:
one = Counter()
print(f"count after one: {Counter.count} = {one.count}.")
two = Counter()
print(f"count after two: {Counter.count} = {one.count} = {two.count}.")

- En **instansvariabel vil overskrive klassevariabler** med samme navn

In [ ]:
two.count = 13
print(f"Class variable overwritten: {Counter.count} = {one.count} != {two.count}.")

Klassevariabelen forblir uendret når den blir overskrevet av en instansvariabel. Overskriving påvirker derfor kun den instansen det gjelder.

## Metoder

Metoder er funksjoner som er del av en klasse. 

Vi skal gå gjennom:
- **Magiske metoder**
- **Statiske metoder**
- **Klassemetoder**

### Hva skjer egentlig når en metode kalles på?

Si at vi har en instans av en gitt klasse:
```Python
instance = GenericClass()
```

Og vi så kaller på en tilhørende metode:
```Python
instance.some_method(args)
```

I bakgrunnen skjer da:
```Python
GenericClass.some_method(instance, args)
```

Derfor bruker vi `self`!

**`Cartesian` som eksempelklasse**: Implementasjon av kartesiske koordinater.

In [ ]:
%%writefile cartesian.py
import math


class Cartesian:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def length(self):
        x, y, z = self.x, self.y, self.z
        return math.sqrt(x*x + y*y + z*z)

**Litt test-dreven utvikling:** Metodene vi implementerer skal defineres av testfunksjoner.

In [ ]:
%%writefile test_cartesian.py
import math
import pytest
from cartesian import Cartesian

ABS_TOL = 1e-9


@pytest.mark.parametrize("x,y,z", [(5, -2, 3), (1, 0, -1)])
def test_fields(x, y, z):
    u = Cartesian(x, y, z)
    assert u.x == x
    assert u.y == y
    assert u.z == z


@pytest.mark.parametrize("x,y,z,length", [
    (2, -2, 1, 3), 
    (3, 0, -4, 5)]
)
def test_length(x, y, z, length):
    u = Cartesian(x, y, z)
    assert math.isclose(u.length(), length, abs_tol=ABS_TOL)

**Implementer `copy` i `Cartesian`:** Metode for å lage kopi av instansen.

- `u.copy()` $\to$ `Cartesian.copy(u)`

In [ ]:
%%writefile --append test_cartesian.py


@pytest.mark.parametrize("x,y,z", [(13, 9, 13), (1, 5, -1)])
def test_copy(x, y, z):
    u = Cartesian(x, y, z)
    v = u.copy()
    assert v.x == x 
    assert v.y == y 
    assert v.z == z
    assert v is not u

**Hint:** Bruk av `type` har fordeler både hvis du endrer navn på klassen og ved arv.

Funksjonen `type` brukes til å finne ut hvilken klasse et objekt tilhører:

In [ ]:
one_int = 1
two_str = "2"
print(f"{type(one_int) = }\n{type(two_str) = }")

`type` kan også brukes til å lage et nytt objekt:

In [ ]:
two_int = type(one_int)(two_str)
print(f"{two_int = } is type {type(two_int)}")

En metode for å lage en kopi kan da se slik ut:

In [ ]:
%%writefile --append cartesian.py

    def copy(self):
        return type(self)(self.x, self.y, self.z)

**Implementer `dot` i `Cartesian`:** Metode for å regne ut dot-produkt.

$$
    u \cdot v = u_x v_x + u_y v_y + u_z v_z
$$

- `u.dot(v)` $\to$ `Cartesian.dot(u, v)`

Konvensjon å bruke `self` ($u$) og `other` ($v$) i implementasjon av metoder!

In [ ]:
%%writefile --append test_cartesian.py


@pytest.mark.parametrize("r_u,r_v,c", [
    ((1, 1, 1), (2, 3, -5), 0),
    ((1, 0, 0), (2, 3, -5), 2),
    ((0, 1, 0), (2, 3, -5), 3),
    ((0, 0, 1), (2, 3, -5), -5)]
)
def test_dot(r_u, r_v, c):
    u = Cartesian(*r_u)
    v = Cartesian(*r_v)
    assert math.isclose(u.dot(v), c, abs_tol=ABS_TOL)

Implementasjon av dot-produkt kan se slik ut:

In [ ]:
%%writefile --append cartesian.py

    def dot(self, other):
        return self.x*other.x + self.y*other.y + self.z*other.z

### Magiske metoder

- Metoder med predefinert funksjon i Python
- Har navn med to understreker foran og bak metodenavnet: `__<magic_method>__`
- Kalles ikke på direkte. Eksempler:
    - Innebygde funksjoner: `str()`, `len()`, osv.
    - Binære operatorer: `+`, `%`, `==`, osv.
    - Indeksering: `a[key]`

Det finnes mange magiske metoder, [se oversikt](https://diveintopython3.net/special-method-names.html). Vi skal se på noen få av dem!

**Implementer `__str__` i `Cartesian`:** Metode for bruk av `str()`.

- `str(u)` $\to$ `u.__str__()` $\to$ `Cartesian.__str__(u)`

Dette brukes automatisk i `print()`!

En enkel implementasjon er gitt i cellen under.

In [ ]:
%%writefile --append cartesian.py

    def __str__(self):
        return f"({self.x:g}, {self.y:g}, {self.z:g})"

Kjør **etter** metoden er implementert:

In [ ]:
from cartesian import Cartesian

u = Cartesian(13, 42, 3.14)
print(u)

**Implementer `__repr__` i `Cartesian`:** Metode for bruk av `repr()`.

- `repr(u)` $\to$ `u.__repr__()` $\to$ `Cartesian.__repr__(u)`

Dette kan f.eks brukes til å lagre et objekt i en tekstfil. 

In [ ]:
%%writefile --append test_cartesian.py


@pytest.mark.parametrize("x,y,z", [(42, -13, 0), (-2, 6, 4)])
def test_repr(x, y, z):
    u_repr = repr(Cartesian(x, y, z))
    assert isinstance(u_repr, str)
    v = eval(u_repr)
    assert isinstance(v, Cartesian)
    assert v.x == x 
    assert v.y == y 
    assert v.z == z

**Hint:** Bruk `type(instance).__name__`.

In [ ]:
print(f"Difference between {type(2) = } and {type(2).__name__ = }.")

Da kan implementasjonen se slik ut:

In [ ]:
%%writefile --append cartesian.py

    def __repr__(self):
        return f"{type(self).__name__}({self.x}, {self.y}, {self.z})"

#### Sammenligninger

Magiske metoder for sammenligninger:
- `__eq__(self, other)`: `a == b`
- `__ne__(self, other)`: `a != b`
- `__lt__(self, other)`: `a < b`
- `__gt__(self, other)`: `a > b`
- `__le__(self, other)`: `a <= b`
- `__ge__(self, other)`: `a >= b`

Hvor `a` er `self` og `b` er `other`.

**Implementer `__eq__` i `Cartesian`:** Metode for evaluering av `u == v`.

- `u == v` $\to$ `u.__eq__(v)` $\to$ `Cartesian.__eq__(u, v)`

Implementerer du alle magiske metoder for sammenligningene, kan man bruke innebygde `sort()` for å sortere lister som inneholder instanser av `Cartesian`.

In [ ]:
%%writefile --append test_cartesian.py


@pytest.mark.parametrize("x,y,z", [(1, 2, 3), (-1, 0, 1)])
def test_eq_copy(x, y, z):
    u = Cartesian(x, y, z)
    v = u.copy()
    assert u == v
    assert u is not v 

@pytest.mark.parametrize("c", [([1],), (1,), ("1",)])  
def test_eq_raises_TypeError(c):
    u = Cartesian(1, 0, -1)
    with pytest.raises(TypeError):
        u == c

Dette kan f.eks se slik ut:

In [ ]:
%%writefile --append cartesian.py

    def __eq__(self, other):
        if not isinstance(other, Cartesian):
            msg = f"Cannot compare {type(self).__name__} with {type(other).__name__}."
            raise TypeError(msg)

        return (self.x, self.y, self.z) == (other.x, other.y, other.z)

#### Aritmetikk

Noen eksempler på magiske metoder for matematikk:
- `__add__(self, other)`: `a + b`
- `__sub__(self, other)`: `a - b`
- `__mul__(self, other)`: `a*b`
- `__pow__(self, other)`: `a**b`
- `__truediv__(self, other)`: `a/b`
- `__floordiv__(self, other)`: `a//b`

Hvor `a` er `self` og `b` er `other`.

**Implementer `__add__` i `Cartesian`:** Metode for evaluering av `u + v`.

- `u + v` $\to$ `u.__add__(v)` $\to$ `Cartesian.__add__(u, v)`

Hvis $w = u + v$, så er $w$ gitt av:
$$
    w_x = u_x + v_x, \\
    w_y = u_y + v_y, \\
    w_z = u_z + v_z.
$$

In [ ]:
%%writefile --append test_cartesian.py


@pytest.mark.parametrize("r_u,r_v,r_w", [
    ((1, 1, 1), (2, 3, -5), (3, 4, -4)),
    ((1, -2, 3), (-2, 3, -5), (-1, 1, -2))]
)
def test_add(r_u, r_v, r_w):
    u = Cartesian(*r_u) 
    v = Cartesian(*r_v)
    w = Cartesian(*r_w)
    assert (u + v) == w

@pytest.mark.parametrize("c", [(2,), ("2",), ([2],)])  
def test_add_raises_TypeError(c):
    u = Cartesian(1, 0, -1)
    with pytest.raises(TypeError):
        u + c

Metoden er implementert i cellen under:

In [ ]:
%%writefile --append cartesian.py

    def __add__(self, other):
        if not isinstance(other, Cartesian):
            msg = f"Cannot add {type(self).__name__} with {type(other).__name__}."
            raise TypeError(msg)
            
        x = self.x + other.x
        y = self.y + other.y
        z = self.z + other.z
        return Cartesian(x, y, z)

**Implementer `__mul__` i `Cartesian`:** Metode for evaluering av `u*v`.

- `u*v` $\to$ `u.__mul__(v)` $\to$ `Cartesian.__mul__(u, v)`

Men la `u*v` være dot-produktet, $u \cdot v$.

In [ ]:
%%writefile --append test_cartesian.py


@pytest.mark.parametrize("r_u,r_v,c", [
    ((2, -4, 2), (1, 1, 1), 0),
    ((8, -2, 3), (2, 3, -3), 1)]
)
def test_mul(r_u, r_v, c):
    u = Cartesian(*r_u) 
    v = Cartesian(*r_v)
    assert u*v == c

@pytest.mark.parametrize("r_u,a,r_w", [
    ((1, 2, 3), 2, (2, 4, 6)),
    ((1, -2, 3), -1, (-1, 2, -3))]
)
def test_mul_scalar(r_u, a, r_w):
    u = Cartesian(*r_u) 
    w = Cartesian(*r_w)
    assert u*a == w
    assert a*u == w

@pytest.mark.parametrize("c", [("2",), ([2],)])  
def test_mul_raises_TypeError(c):
    u = Cartesian(1, 0, -1)
    with pytest.raises(TypeError):
        u*c

Implementasjonen kan da se ut som kodesnutten under. Én av testene vil likevel ikke passere. Kan du se hvorfor?

In [ ]:
%%writefile --append cartesian.py

    def __mul__(self, other):
        """Interpret u*v to be the dot product."""
        if isinstance(other, Cartesian):
            return self.dot(other)
        elif isinstance(other, (int, float)):
            return Cartesian(self.x*other, self.y*other, self.z*other)
        else:
            msg = f"Cannot multiply {type(self).__name__} with {type(other).__name__}."
            raise TypeError(msg)

#### Aritmetikk fra motsatt side

Hvis aritmetikk-operatorer kaster et unntak, forsøkes det med *reflected* operatorer:
- `__radd__(self, other)`: `b + a`
- `__rsub__(self, other)`: `b - a`
- `__rmul__(self, other)`: `b*a`
- `__rtruediv__(self, other)`: `b/a`
- `__rfloordiv__(self, other)`: `b//a`
- `__rpow__(self, other)`: `b**a`

Hvor `a` er `self` og `b` er `other`.

**Implementer `__rmul__` i `Cartesian`:** Metode for evaluering av `c*u`.

- `c*u` $\to$ `u.__rmul__(c)` $\to$ `Cartesian.__rmul__(u, c)`


En enkel løsning:

In [ ]:
%%writefile --append cartesian.py

    def __rmul__(self, other):
        return self*other

#### Aritmetikk som skjer *in-place*

Noen eksempler:
- `__iadd__(self, other)`: `a += b`
- `__isub__(self, other)`: `a -= b`
- `__imul__(self, other)`: `a *= b`
- `__itruediv__(self, other)`: `a /= b`
- `__ifloordiv__(self, other)`: `a //= b`
- `__ipow__(self, other)`: `a **= b`

Hvor `a` er `self` og `b` er `other`.

#### Andre magiske metoder

Det finnes [mange andre magiske metoder](https://diveintopython3.net/special-method-names.html). Her er et lite utvalg til:

- `__neg__(self)`: `-a`
- `__iter__(self)`: `for x in a:`
- `__len__`: `len(a)`
- `__contains__(self, value)`: `value in a`
- `__getitem__(self, key)`: `a[key]`
- `__setitem__(self, key, value)`: `a[key] = value`

Hvor `a` er `self`.

### Statiske metoder

- Bruker dekorator: `@staticmethod`
- Statiske metoder **avhenger verken av klassen eller instansen**
- Hvis du trenger `self` eller `cls` er dette **ikke** en statisk metode
- Brukes når metoden *passer inn* i en klasse

```Python
class SomeClass:
    @staticmethod
    def static_method(args):
         ...
```

Statiske metoder kan brukes uten å opprette en instans: `SomeClass.static_method(args)`

**Implementer statisk metode i `Cartesian`:** `spherical_to_cartesian(r, theta, phi) -> (x, y z)` 

Kulekoordinater er gitt av avstanden $r$, den polare vinkelen $\theta \in [0, \pi]$ og azimutal vinkel $\varphi \in [0, 2\pi]$$. Oversatt til kartesisk koordinatsystem:

$$
x = r\sin(\theta)\cos(\varphi), \\
y = r\sin(\theta)\sin(\varphi), \\
z = r\cos(\theta).
$$

In [ ]:
%%writefile --append test_cartesian.py


@pytest.mark.parametrize("spherical,cartesian", [
    ((-1, 0, 1), (0, 0, -1)),
    ((2, math.pi/4, math.pi), (-math.sqrt(2), 0, math.sqrt(2)))]
)
def test_spherical_to_cartesian(spherical, cartesian):
    r, theta, phi = spherical
    x, y, z = Cartesian.spherical_to_cartesian(r, theta, phi)
    assert math.isclose(x, cartesian[0], abs_tol=ABS_TOL)
    assert math.isclose(y, cartesian[1], abs_tol=ABS_TOL)
    assert math.isclose(z, cartesian[2], abs_tol=ABS_TOL)
    

Her er da den statiske metoden:

In [ ]:
%%writefile --append cartesian.py
  
    @staticmethod
    def spherical_to_cartesian(r, theta, phi):
        x = r*math.sin(theta)*math.cos(phi)
        y = r*math.sin(theta)*math.sin(phi)
        z = r*math.cos(theta)
        return (x, y, z)

### Klassemetoder

- Bruker dekorator: `@classmethod`
- Brukes når **metoden avhenger av klassen**, men ikke av instansen
- Hvis du trenger `self`, skal dette **ikke** være en klassemetode
- Klassemetoder bruker `cls` som første argument

```Python
class SomeClass:
    @classmethod
    def class_method(cls, args):
        ...
```
Klassemetoder kan brukes uten å opprette en instans: `SomeClass.class_method(args)`

#### *Factory*-metoder

*Factory* er en klassemetode som gir en alternativ måte å opprette en instans på.

```Python
class SomeClass:
    def __init__(self, args):
        ...
        
    @classmethod
    def factory(cls, factory_args):
        # convert factory args to suitable arguments for init
        args = ...
        return cls(args)
```

**Implementer factory-metode til `Cartesian`:** Opprett instans basert på kulekoordinater.

- `Cartesian.from_spherical(r, theta, phi)` $\to$ `Cartesian.from_spherical(Cartesian, r, theta, phi)`

In [ ]:
%%writefile --append test_cartesian.py


@pytest.mark.parametrize("spherical,cartesian", [
    ((42, math.pi, 13), (0, 0, -42)),
    ((-2, math.pi/4, math.pi), (math.sqrt(2), 0, -math.sqrt(2)))]
)
def test_factory(spherical, cartesian):
    u = Cartesian.from_spherical(*spherical)
    v = Cartesian(*cartesian)
    assert math.isclose(u.x, v.x, abs_tol=ABS_TOL)
    assert math.isclose(u.y, v.y, abs_tol=ABS_TOL)
    assert math.isclose(u.z, v.z, abs_tol=ABS_TOL)

Factory-metode til `Cartesian` ser da slik ut:

In [ ]:
%%writefile --append cartesian.py

    @classmethod
    def from_spherical(cls, r, theta, phi):
        x, y, z = cls.spherical_to_cartesian(r, theta, phi)
        return cls(x, y, z)

## Attributter i Python

Vi har snakket om:
- **Datafelt**
    - Instansvariabler
    - Klassevariabler
- **Metoder**
    - *normale* metoder
    - Magiske metoder
    - Klassemetoder
    - Statiske metoder
    
Samt **private** attributter.